<a href="https://colab.research.google.com/github/xjdeng/hyperliquid_testnet_trading/blob/main/hyperliquid_momentum%2Bsentiment_strat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Setup

In [1]:
!pip install hyperliquid-python-sdk duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.7/580.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.3/170.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.0 MB/s eta 0:00:00


In [2]:
from eth_account import Account
from hyperliquid.exchange import Exchange
from hyperliquid.info import Info
from hyperliquid.utils import constants
from google.colab import userdata
import logging
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import time
from duckduckgo_search import DDGS
import enum
import json
from typing_extensions import TypedDict
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
assert GOOGLE_API_KEY is not None
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash-latest")


private_key = userdata.get("hypernet_private_key")
account = Account.from_key(private_key)
address = account.address

exchange = Exchange(account, constants.TESTNET_API_URL)
info = Info(constants.TESTNET_API_URL)
# Query balance
balance_info = info.user_state(account.address)
print(balance_info)

{'marginSummary': {'accountValue': '991.047217', 'totalNtlPos': '891.445974', 'totalRawUsd': '99.601243', 'totalMarginUsed': '178.059713'}, 'crossMarginSummary': {'accountValue': '958.282776', 'totalNtlPos': '792.239462', 'totalRawUsd': '166.043314', 'totalMarginUsed': '145.295272'}, 'crossMaintenanceMarginUsed': '72.647635', 'withdrawable': '773.426491', 'assetPositions': [{'type': 'oneWay', 'position': {'coin': 'AAVE', 'szi': '0.53', 'leverage': {'type': 'cross', 'value': 20}, 'entryPx': '184.68', 'positionValue': '97.3345', 'unrealizedPnl': '-0.5459', 'returnOnEquity': '-0.1115442928', 'liquidationPx': None, 'marginUsed': '4.866725', 'maxLeverage': 20, 'cumFunding': {'allTime': '0.0', 'sinceOpen': '0.0', 'sinceChange': '0.0'}}}, {'type': 'oneWay', 'position': {'coin': 'LDO', 'szi': '98.7', 'leverage': {'type': 'cross', 'value': 20}, 'entryPx': '1.0325', 'positionValue': '101.13789', 'unrealizedPnl': '-0.76986', 'returnOnEquity': '-0.1510895884', 'liquidationPx': None, 'marginUsed': 

In [3]:
meta, asset_ctxs = info.meta_and_asset_ctxs()

In [4]:
# Combine names with their associated liquidity data
market_stats = [
    {
        "name": m["name"],
        "dayNtlVlm": float(a["dayNtlVlm"]),
        "openInterest": float(a["openInterest"]),
        "impactPxs": a.get("impactPxs", None)
    }
    for m, a in zip(meta["universe"], asset_ctxs)
]

# Sort by 24h volume (descending) and take top 10
top_markets_all = sorted(market_stats, key=lambda x: x["dayNtlVlm"], reverse=True)
top_markets = top_markets_all[:100]

In [5]:
len(top_markets_all)

164

In [6]:


def get_hourly_candles(info, coin, lookback_hours=24):
    now = datetime.utcnow()
    start = now - timedelta(hours=lookback_hours)
    start_ts = int(start.timestamp() * 1000)
    end_ts = int(now.timestamp() * 1000)

    candles = info.candles_snapshot(
        name=coin,
        interval="1h",
        startTime=start_ts,
        endTime=end_ts
    )
    return candles

def prepare_ohlc_points(candles):
    # Each candle has o, h, l, c
    all_points = []
    for candle in candles:
        o, h, l, c = float(candle["o"]), float(candle["h"]), float(candle["l"]), float(candle["c"])
        all_points.extend([o, h, l, c])  # 4 points per bar
    return np.array(all_points)

def compute_slope_std_ratio(y_vals):
    x_vals = np.arange(len(y_vals))
    A = np.vstack([x_vals, np.ones_like(x_vals)]).T
    slope, intercept = np.linalg.lstsq(A, y_vals, rcond=None)[0]
    y_pred = slope * x_vals + intercept
    residuals = y_vals - y_pred
    std_dev = np.std(residuals)
    if std_dev == 0:
        return float("-inf")
    return slope / std_dev

In [7]:


# Function to fetch 24 hourly candles for a given coin
def get_hourly_candles(info, coin, lookback_hours=24):
    now = datetime.utcnow()
    start = now - timedelta(hours=lookback_hours)
    start_ts = int(start.timestamp() * 1000)
    end_ts = int(now.timestamp() * 1000)
    # Use the candles_snapshot method to fetch data
    candles = info.candles_snapshot(name=coin, interval="1h", startTime=start_ts, endTime=end_ts)
    return candles

# Prepare OHLC points: extract open, high, low, close from each candle (4 points per candle)
def prepare_ohlc_points(candles):
    all_points = []
    for candle in candles:
        try:
            o = float(candle["o"])
            h = float(candle["h"])
            l = float(candle["l"])
            c = float(candle["c"])
            all_points.extend([o, h, l, c])
        except Exception as e:
            print("Error processing candle:", candle, e)
    return np.array(all_points)

# Compute regression slope and standard deviation of residuals, then return slope/std
def compute_slope_std_ratio(y_vals):
    x_vals = np.arange(len(y_vals))
    A = np.vstack([x_vals, np.ones_like(x_vals)]).T
    slope, intercept = np.linalg.lstsq(A, y_vals, rcond=None)[0]
    y_pred = slope * x_vals + intercept
    residuals = y_vals - y_pred
    std_dev = np.std(residuals)
    if std_dev == 0:
        return float("-inf")
    return slope / std_dev

# Robust wrapper to fetch candles with unlimited retries for HTTP 429 errors
def safe_get_candles(info, coin, initial_delay=1, max_delay=60):
    delay = initial_delay
    attempt = 0
    while True:
        try:
            candles = get_hourly_candles(info, coin)
            return candles
        except Exception as e:
            # Try to extract error code from exception attributes or message
            code = None
            if hasattr(e, "status_code"):
                code = e.status_code
            elif isinstance(e, tuple) and len(e) > 0:
                code = e[0]
            elif "429" in str(e):
                code = 429

            if code == 429:
                attempt += 1
                print(f"Rate limit hit for {coin}, retrying in {delay} seconds... (attempt {attempt})")
                time.sleep(delay)
                # Exponential backoff with cap
                delay = min(delay * 2, max_delay)
            else:
                raise e

# Main loop: iterate over the top_markets (top 100 securities) and compute the regression-based score.
signal_scores = []

for market in top_markets:
    name = market["name"]
    try:
        candles = safe_get_candles(info, name)
        if len(candles) < 24:
            print(f"Not enough data for {name}.")
            continue  # skip if insufficient data
        points = prepare_ohlc_points(candles)  # 96 points total (24 bars * 4 OHLC points)
        if len(points) == 0:
            continue
        score = compute_slope_std_ratio(points)
        signal_scores.append({
            "name": name,
            "score": score
        })
    except Exception as e:
        print(f"Error processing {name}: {e}")

# Sort the watchlist by the computed score (slope / volatility ratio) in descending order
ranked_markets = sorted(signal_scores, key=lambda x: x["score"], reverse=True)
print("Ranked Markets (top 10):")
for i, market in enumerate(ranked_markets[:10], 1):
    print(f"{i}. {market['name']} - Score: {market['score']:.4f}")


Rate limit hit for HBAR, retrying in 1 seconds... (attempt 1)
Rate limit hit for HBAR, retrying in 2 seconds... (attempt 2)
Rate limit hit for HBAR, retrying in 4 seconds... (attempt 3)
Rate limit hit for HBAR, retrying in 8 seconds... (attempt 4)
Rate limit hit for HBAR, retrying in 16 seconds... (attempt 5)
Rate limit hit for HBAR, retrying in 32 seconds... (attempt 6)
Ranked Markets (top 10):
1. JUP - Score: 0.0682
2. AAVE - Score: 0.0620
3. GRASS - Score: 0.0615
4. W - Score: 0.0607
5. BLUR - Score: 0.0560
6. SOL - Score: 0.0558
7. ZK - Score: 0.0520
8. SCR - Score: 0.0514
9. SPX - Score: 0.0499
10. OM - Score: 0.0453


In [8]:


def get_crypto_news_text(ticker, max_results=50):
    query = f"{ticker} crypto news"
    results = []

    with DDGS() as ddgs:
        for r in ddgs.news(query, max_results=max_results):
            title = r.get("title", "").strip()
            body = r.get("body", "").strip()
            url = r.get("url", "").strip()
            if title:
                results.append(f"{title}\n{body}\n{url}\n")

    return "\n".join(results)

In [9]:
news_text = get_crypto_news_text("SOL")
print(news_text[:1000])  # print first 1000 characters

Not Solana (SOL), Not Dogecoin (DOGE) - This 1000x Contender Is Tipped To Be The Next Crypto To Explode
While Solana (SOL) and Dogecoin (DOGE) have dominated the crypto space, a new 1000x contender is gaining momentum and could be the next big breakout. Lightchain AI has already raised over $18.1 million at a presale price of $0.
https://techbullion.com/not-solana-sol-not-dogecoin-doge-this-1000x-contender-is-tipped-to-be-the-next-crypto-to-explode/

Is This Layer 1 Blockchain the Best Altcoin to Buy This Month Out of These 8 Top Picks? Crypto Experts Weigh In
Have you been wondering which altcoins are worth adding to your portfolio this month? With the market constantly evolving, selecting the right projects can be the key to unlocking significant gains. Crypto experts are keeping a close eye on projects that offer real utility,
https://www.msn.com/en-us/technology/cryptocurrencies/is-this-layer-1-blockchain-the-best-altcoin-to-buy-this-month-out-of-these-8-top-picks-crypto-experts-we

In [10]:
len(news_text)

19468

In [11]:


class Sentiment(enum.Enum):
  negative = "negative"
  neutral = "neutral"
  positive = "positive"

class Confidence(enum.Enum):
  low = "low"
  medium = "medium"
  high = "high"

class Result(TypedDict):
    sentiment: Sentiment
    confidence: Confidence
    explanation: str

def predict_sentiment_llm(txt, crypto):
    prompt = f"""
    Gauge the overall sentiment of the Duckduckgo Search Results below about a particular cryptocurrency: {crypto}.
    If there are both positive and negative sentiment articles in the mix, pay attention to which side seems to be more dominant.
    Weight more recent news more than older news.
    If neither positive or negative news is clearly dominant, then the sentiment is neutral
    Please also state your confidence in the prediction and give an explanation for the type you chose and your confidence in it.

    Search Results:
    ---
    {txt}
    ---
    """
    result = model.generate_content(prompt,
                                    generation_config = genai.GenerationConfig(
                                        response_mime_type="application/json", response_schema=Result
                                    ))
    return json.loads(result.to_dict()['candidates'][0]['content']['parts'][0]['text'])

def predict_sentiment(crypto):
  news = get_crypto_news_text(crypto)
  #print(news[0:500])
  return predict_sentiment_llm(news, crypto)

In [12]:
predict_sentiment("STEPN")

{'confidence': 'medium',
 'explanation': "While there is some positive news regarding STEPN's price increase and a bullish prediction, the majority of the search results focus on negative events in the broader crypto market such as major hacks and heists, regulatory issues, and the overall uncertainty associated with the crypto industry.  The recent positive news about STEPN is overshadowed by the prevalence of negative crypto news, thus leading to an overall negative sentiment, although the confidence is only medium due to the limited number of direct STEPN-related articles compared to broader crypto market news.",
 'sentiment': 'negative'}

In [13]:
def get_final_picks(original_list, min_ratio = 0, n = 10):
  resulting_list = []
  for coin in original_list:
    if coin['score'] < min_ratio:
      continue
    sentiment = predict_sentiment(coin['name'])
    if sentiment['sentiment'] != "negative":
      resulting_list.append(coin['name'])
      if len(resulting_list) >= n:
        break
  missing = n - len(resulting_list)
  for _ in range(missing):
    resulting_list.append("USDC")
  return resulting_list

In [14]:
final_picks = get_final_picks(ranked_markets)

In [15]:
from decimal import Decimal, ROUND_DOWN
from datetime import datetime, timedelta
from math import floor


# Reconfigure logging to ensure output appears.
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")
logging.debug("Logging is working")

def adjust_order_size(coin: str, proposed_size: float, info) -> float:
    """
    Retrieves the minimum order increment for the given coin from metadata,
    then rounds the proposed_size down to the nearest multiple of that increment.

    Returns the adjusted order size.
    """
    try:
        meta = info.meta()
    except Exception as e:
        logging.error(f"Error retrieving metadata: {e}")
        return proposed_size
    universe = meta.get("universe", [])
    for asset in universe:
        if asset.get("name", "").upper() == coin.upper():
            sz_decimals = asset.get("szDecimals", 8)
            min_increment = 1 / (10 ** sz_decimals)
            # Floor the proposed size to a multiple of min_increment.
            adjusted_size = floor(proposed_size / min_increment) * min_increment
            logging.debug(f"For {coin}: proposed_size={proposed_size} min_increment={min_increment:.8f} adjusted_size={adjusted_size}")
            return adjusted_size
    logging.error(f"Coin {coin} not found in metadata. Using proposed size {proposed_size}.")
    return proposed_size

def sell_all_to_usdc(exchange, info):
    """
    Closes all open positions for every asset held by the account.
    For each unique coin in info.user_state(), it calls exchange.market_close(coin)
    to liquidate the position.
    """
    address = exchange.wallet.address
    state = info.user_state(address)
    positions = state.get("assetPositions", [])
    if not positions:
        logging.info("No positions to close.")
        return

    # Build a set of unique coins with positions.
    coins = set()
    for pos in positions:
        coin = pos.get("position", {}).get("coin")
        if coin:
            coins.add(coin)

    for coin in coins:
        logging.info(f"Attempting to market close all positions for {coin}.")
        order_result = exchange.market_close(coin)
        if order_result and order_result.get("status") == "ok":
            for status in order_result["response"]["data"]["statuses"]:
                try:
                    filled = status["filled"]
                    logging.info(f"Market close order #{filled['oid']} for {coin} filled {filled['totalSz']} @{filled['avgPx']}")
                except KeyError:
                    logging.error(f"Error closing position for {coin}: {status.get('error')}")
        else:
            logging.error(f"Market close failed for {coin}: {order_result}")
        time.sleep(2)  # small pause between orders

def rebalance_equal_weight(exchange, info, picks, slippage=0.01):
    """
    Rebalances the portfolio so that the provided list of coin tickers (picks)
    becomes equal-weighted.

    Steps:
      1. Sell all existing positions via market_close.
      2. Query available USDC from info.user_state() using exchange.wallet.address.
      3. Divide USDC equally among all entries in picks.
      4. For each coin in picks, compute the order size as:
             size = (allocated USDC) / (mid price from info.all_mids())
         Then adjust the size down to the coin's minimum increment using adjust_order_size(),
         and place a market BUY order via market_open.
         The last order uses the remaining USDC.
    """
    logging.info("Starting portfolio rebalance process.")

    # Step 1: Liquidate all positions.
    logging.info("Closing all open positions to convert to USDC.")
    sell_all_to_usdc(exchange, info)

    # Step 2: Retrieve the available USDC balance.
    address = exchange.wallet.address
    state = info.user_state(address)
    balance_summary = state.get("crossMarginSummary", {})
    try:
        usdc_balance = float(balance_summary.get("accountValue", 0))
    except Exception as e:
        raise ValueError(f"Error parsing USDC balance: {e}")
    if usdc_balance <= 0:
        raise ValueError("No USDC available for allocation.")

    total_slots = len(picks)
    if total_slots == 0:
        logging.info("No picks provided; nothing to allocate.")
        return

    allocation_per_slot = usdc_balance / total_slots
    logging.info(f"Total USDC: {usdc_balance:.2f}. Allocating ~{allocation_per_slot:.2f} per slot.")

    # Step 3: Get current mid prices.
    mids = info.all_mids()

    # Step 4: Place market BUY orders.
    for i, coin in enumerate(picks):
        # For all but the last pick, allocate a fixed amount; last uses the remaining balance.
        notional = allocation_per_slot if i < total_slots - 1 else usdc_balance
        try:
            mid_price = float(mids[coin])
        except Exception as e:
            logging.error(f"Error retrieving mid price for {coin}: {e}")
            continue

        # Compute the initial order size from the notional and mid price.
        proposed_size = notional / mid_price

        # Adjust the proposed size to the coin's minimum increment.
        adjusted_size = adjust_order_size(coin, proposed_size, info)
        if adjusted_size <= 0:
            logging.error(f"Adjusted order size for {coin} is zero. Skipping order.")
            continue

        logging.info(f"Placing market BUY order for {coin}: proposed_size={proposed_size:.8f}, adjusted_size={adjusted_size:.8f} (notional {notional:.2f} / mid price {mid_price:.2f}), slippage={slippage}")

        order_result = exchange.market_open(coin, True, adjusted_size, None, slippage)
        if order_result and order_result.get("status") == "ok":
            for status in order_result["response"]["data"]["statuses"]:
                try:
                    filled = status["filled"]
                    logging.info(f"Market open order #{filled['oid']} for {coin} filled {filled['totalSz']} @{filled['avgPx']}")
                except KeyError:
                    logging.error(f"Error with market open order for {coin}: {status.get('error')}")
        else:
            logging.error(f"Market open order failed for {coin}: {order_result}")

        # Update remaining USDC balance (approximation using mid price).
        usdc_balance -= adjusted_size * mid_price
        logging.info(f"Remaining USDC balance: {usdc_balance:.2f}")
        time.sleep(2)  # pause between orders

def get_portfolio_positions(exchange, info):
    """
    Retrieves portfolio positions for the current wallet on Hyperliquid.
    Returns a list of dictionaries with key details for each position.
    """
    address = exchange.wallet.address
    state = info.user_state(address)
    logging.info(f"User state for {address}: {state}")
    positions = state.get("assetPositions", [])

    formatted_positions = []
    for pos in positions:
        position_data = pos.get("position", {})
        formatted_positions.append({
            "coin": position_data.get("coin"),
            "entry_price": position_data.get("entryPx"),
            "leverage": position_data.get("leverage"),
            "liquidation_price": position_data.get("liquidationPx"),
            "margin_used": position_data.get("marginUsed"),
            "position_value": position_data.get("positionValue"),
            "return_on_equity": position_data.get("returnOnEquity"),
            "size": position_data.get("szi"),
            "unrealized_pnl": position_data.get("unrealizedPnl")
        })

    return formatted_positions

2025-03-23 15:02:02,761 - DEBUG - Logging is working


In [16]:
sell_all_to_usdc(exchange, info)

2025-03-23 15:02:02,939 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 3558
2025-03-23 15:02:02,940 - INFO - Attempting to market close all positions for JUP.
2025-03-23 15:02:02,943 - DEBUG - Resetting dropped connection: api.hyperliquid-testnet.xyz
2025-03-23 15:02:03,519 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 3558
2025-03-23 15:02:03,691 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 22828
2025-03-23 15:02:03,864 - DEBUG - {'action': {'type': 'order', 'orders': [{'a': 71, 'b': False, 'p': '0.52382', 's': '179', 'r': True, 't': {'limit': {'tif': 'Ioc'}}}], 'grouping': 'na'}, 'nonce': 1742742123857, 'signature': {'r': '0x5c1229b15e255c222467fdedea5f6b4d03e844aeb6cfc53f17f75c1386eea3e8', 's': '0x4aa24eb626ebb66eeab6dd7ef26217606b0d403b48033dcb586a26a9f318bda5', 'v': 28}, 'vaultAddress': None}
2025-03-23 15:02:03,867 - DEBUG - Starting new HTTPS connection (1): api.hyperliquid-testnet.xyz:443


In [17]:
rebalance_equal_weight(exchange, info, final_picks)

2025-03-23 15:02:38,010 - INFO - Starting portfolio rebalance process.
2025-03-23 15:02:38,011 - INFO - Closing all open positions to convert to USDC.
2025-03-23 15:02:38,179 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 344
2025-03-23 15:02:38,180 - INFO - No positions to close.
2025-03-23 15:02:38,349 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 344
2025-03-23 15:02:38,350 - INFO - Total USDC: 990.52. Allocating ~99.05 per slot.
2025-03-23 15:02:38,685 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 22830
2025-03-23 15:02:38,855 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 8347
2025-03-23 15:02:38,856 - DEBUG - For JUP: proposed_size=179.64462752210386 min_increment=1.00000000 adjusted_size=179.0
2025-03-23 15:02:38,857 - INFO - Placing market BUY order for JUP: proposed_size=179.64462752, adjusted_size=179.00000000 (notional 99.05 / mid price 0.55), slippage=0.01
2

In [18]:
get_portfolio_positions(exchange, info)

2025-03-23 15:03:17,517 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 3790
2025-03-23 15:03:17,518 - INFO - User state for 0x915d30477E3Ae7621a6c4b5dfeEC88b0d6728BD8: {'marginSummary': {'accountValue': '989.400213', 'totalNtlPos': '990.000602', 'totalRawUsd': '-0.600389', 'totalMarginUsed': '224.588835'}, 'crossMarginSummary': {'accountValue': '923.60708', 'totalNtlPos': '792.171983', 'totalRawUsd': '131.435097', 'totalMarginUsed': '158.795702'}, 'crossMaintenanceMarginUsed': '79.281141', 'withdrawable': '725.627274', 'assetPositions': [{'type': 'oneWay', 'position': {'coin': 'SOL', 'szi': '0.74', 'leverage': {'type': 'cross', 'value': 20}, 'entryPx': '132.51', 'positionValue': '98.0352', 'unrealizedPnl': '-0.0222', 'returnOnEquity': '-0.0045279602', 'liquidationPx': None, 'marginUsed': '4.90176', 'maxLeverage': 21, 'cumFunding': {'allTime': '0.0', 'sinceOpen': '0.0', 'sinceChange': '0.0'}}}, {'type': 'oneWay', 'position': {'coin': 'AAVE', 'szi': '0.53', '

[{'coin': 'SOL',
  'entry_price': '132.51',
  'leverage': {'type': 'cross', 'value': 20},
  'liquidation_price': None,
  'margin_used': '4.90176',
  'position_value': '98.0352',
  'return_on_equity': '-0.0045279602',
  'size': '0.74',
  'unrealized_pnl': '-0.0222'},
 {'coin': 'AAVE',
  'entry_price': '183.42',
  'leverage': {'type': 'cross', 'value': 20},
  'liquidation_price': None,
  'margin_used': '4.85745',
  'position_value': '97.149',
  'return_on_equity': '-0.0130847236',
  'size': '0.53',
  'unrealized_pnl': '-0.0636'},
 {'coin': 'BLUR',
  'entry_price': '0.11275',
  'leverage': {'type': 'cross', 'value': 10},
  'liquidation_price': None,
  'margin_used': '9.906474',
  'position_value': '99.06474',
  'return_on_equity': '0.0070953437',
  'size': '878.0',
  'unrealized_pnl': '0.07024'},
 {'coin': 'JUP',
  'entry_price': '0.55166',
  'leverage': {'type': 'cross', 'value': 3},
  'liquidation_price': None,
  'margin_used': '32.890653',
  'position_value': '98.67196',
  'return_on_e

In [19]:
info.user_state(address)

2025-03-23 15:03:17,699 - DEBUG - https://api.hyperliquid-testnet.xyz:443 "POST /info HTTP/1.1" 200 3790


{'marginSummary': {'accountValue': '989.400213',
  'totalNtlPos': '990.000602',
  'totalRawUsd': '-0.600389',
  'totalMarginUsed': '224.588835'},
 'crossMarginSummary': {'accountValue': '923.60708',
  'totalNtlPos': '792.171983',
  'totalRawUsd': '131.435097',
  'totalMarginUsed': '158.795702'},
 'crossMaintenanceMarginUsed': '79.281141',
 'withdrawable': '725.627274',
 'assetPositions': [{'type': 'oneWay',
   'position': {'coin': 'SOL',
    'szi': '0.74',
    'leverage': {'type': 'cross', 'value': 20},
    'entryPx': '132.51',
    'positionValue': '98.0352',
    'unrealizedPnl': '-0.0222',
    'returnOnEquity': '-0.0045279602',
    'liquidationPx': None,
    'marginUsed': '4.90176',
    'maxLeverage': 21,
    'cumFunding': {'allTime': '0.0',
     'sinceOpen': '0.0',
     'sinceChange': '0.0'}}},
  {'type': 'oneWay',
   'position': {'coin': 'AAVE',
    'szi': '0.53',
    'leverage': {'type': 'cross', 'value': 20},
    'entryPx': '183.42',
    'positionValue': '97.149',
    'unrealizedP